In [12]:
import os
os.chdir('c:\\Users\\abbaig\\Krish naik course\\heart-disease-prediction')



In [92]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [93]:
def set_the_header(data_path):
    df = pd.read_csv(data_path)
    df=df.iloc[1:]
    df.columns=df.iloc[0]
    df = df.drop(df.index[0])
    df.reset_index(drop=True)
    return df

def feature_engineering(df):
    
    df[['job','education']]=df['jobedu'].str.split(',',expand=True)
    df[['months','years']]=df.month.str.split(",",expand=True)
    df.drop(['jobedu','month','years'],axis=1,inplace=True)
    df.targeted=df.targeted.map({'yes':1,'no':0})
    df.default=df.default.map({'yes':1,'no':0})
    df.loan=df.loan.map({'yes':1,'no':0})
    df.housing=df.housing.map({'yes':1,'no':0})
    seconds_mask=df['duration'].astype(str).str.endswith('sec')
    df.loc[seconds_mask,'duration']=df.loc[seconds_mask,'duration'].str.replace('sec', '').astype(float) / 60
    minutes_mask=df['duration'].astype(str).str.endswith('min')
    df.loc[minutes_mask,'duration']=df.loc[minutes_mask,'duration'].str.replace('min','').astype(float)
    df.duration=df.duration.astype(float)
    df.drop(['customerid','poutcome'],axis=1,inplace=True)
    df.day=df.day.astype('int64')
    df = df.dropna(subset=['age'])
    df.age=df.age.astype('int64')
    df.salary=df.salary.astype(float)
    df.balance=df.balance.astype(float)
    df.pdays=df.pdays.astype('int64')
    df=df[df['pdays']<500] 
    bins_pdays=[-1,0,50,100,150,200,250,300,350,400,450,500]
    labels_pdays=['never-reached','0-50','50-100','100-150','150-200','200-250','250-300','300-350','350-400','400-450','450-500']
    df['pdays-bin']=pd.cut(df['pdays'],bins=bins_pdays,labels=labels_pdays,right=False)
    df.drop('pdays',axis=1,inplace=True)
    df.previous=df.previous.astype('int64')
    df= df[df['previous']<100]
    bins_previous=[0,10,30,60,90,120]
    labels_previous= ['never answered','10-30','30-60','60-90','90+']
    df['previous-bin']= pd.cut(df['previous'],bins=bins_previous,labels=labels_previous,right=False)
    df.drop(['previous'],axis=1,inplace=True)
    list_of_categorical =['marital','contact','day','campaign','job','education','months','pdays-bin','previous-bin']
    list_of_numerical =['age','salary','balance','duration']
    dummy= pd.get_dummies(df[list_of_categorical],drop_first=True)
    dummy=dummy.astype(int)
    df_one_hot_encoded= pd.concat([df,dummy],axis=1)
    scaler = MinMaxScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df_one_hot_encoded[list_of_numerical]),columns=['age_scaled','salary_scaled','balance_scaled','duration_scaled'])
    df = pd.concat([df_one_hot_encoded,df_scaled],axis=1)
    df.drop(list_of_numerical,axis=1,inplace=True)
    df.drop(list_of_categorical,axis=1,inplace=True)
    return df

In [94]:
df = set_the_header('artifacts/data_ingestion/bank_marketing_updated_v1.csv')
# df = feature_engineering(df)
df = feature_engineering(df)
df.head()

C:\Users\abbaig\AppData\Local\Temp\ipykernel_22944\781479950.py:2: DtypeWarning: Columns (0,1,2,3,11,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)
C:\Users\abbaig\AppData\Local\Temp\ipykernel_22944\781479950.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.age=df.age.astype('int64')
C:\Users\abbaig\AppData\Local\Temp\ipykernel_22944\781479950.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.salary=df.salary.astype(float)
C:\Us

,targeted,default,housing,loan,response,marital_married,marital_single,contact_telephone,contact_unknown,campaign_2,...,pdays-bin_400-450,pdays-bin_450-500,previous-bin_10-30,previous-bin_30-60,previous-bin_60-90,previous-bin_90+,age_scaled,salary_scaled,balance_scaled,duration_scaled
2,1.0,0.0,1.0,0.0,no,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.194805,1.000000,0.072822,0.015453
3,1.0,0.0,1.0,0.0,no,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.376623,0.166667,0.086476,0.018707
4,1.0,0.0,1.0,1.0,no,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.194805,0.000000,0.072812,0.040260
5,0.0,0.0,1.0,0.0,no,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.220779,0.833333,0.074901,0.028264
6,0.0,0.0,0.0,0.0,no,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.129870,0.833333,0.076862,0.044124


In [72]:
#one hot encoding of the variables 


In [81]:
df = pd.concat([df_one_hot_encoded,df_scaled],axis=1)

In [90]:
df.dropna(inplace=True)

In [91]:
df

,targeted,default,housing,loan,response,marital_married,marital_single,contact_telephone,contact_unknown,campaign_2,...,pdays-bin_400-450,pdays-bin_450-500,previous-bin_10-30,previous-bin_30-60,previous-bin_60-90,previous-bin_90+,age_scaled,salary_scaled,balance_scaled,duration_scaled
2,1.0,0.0,1.0,0.0,no,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.194805,1.000000,0.072822,0.015453
3,1.0,0.0,1.0,0.0,no,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.376623,0.166667,0.086476,0.018707
4,1.0,0.0,1.0,1.0,no,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.194805,0.000000,0.072812,0.040260
5,0.0,0.0,1.0,0.0,no,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.220779,0.833333,0.074901,0.028264
6,0.0,0.0,0.0,0.0,no,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.129870,0.833333,0.076862,0.044124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45079,1.0,0.0,0.0,0.0,no,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.428571,0.500000,0.080293,0.198658
45080,1.0,0.0,0.0,0.0,yes,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.688312,0.458333,0.088501,0.092721
45081,0.0,0.0,0.0,0.0,yes,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.701299,0.458333,0.124689,0.229158
45082,1.0,0.0,0.0,0.0,no,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.506494,0.166667,0.078868,0.103294


In [237]:
df = set_the_header('artifacts/data_ingestion/bank_marketing_updated_v1.csv')
# df = feature_engineering(df)
df = feature_engineering(df)
df.head()

C:\Users\abbaig\AppData\Local\Temp\ipykernel_22272\3631738921.py:2: DtypeWarning: Columns (0,1,2,3,11,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


1,customerid,age,salary,balance,marital,targeted,default,housing,loan,contact,day,duration,campaign,pdays,previous,poutcome,response,job,education,months
2,1,58,100000,2143,married,1,0,1,0,unknown,5,4.350000,1,-1,0,unknown,no,management,tertiary,may
3,2,44,60000,29,single,1,0,1,0,unknown,5,2.516667,1,-1,0,unknown,no,technician,secondary,may
4,3,33,120000,2,married,1,0,1,1,unknown,5,1.266667,1,-1,0,unknown,no,entrepreneur,secondary,may
5,4,47,20000,1506,married,0,0,1,0,unknown,5,1.533333,1,-1,0,unknown,no,blue-collar,unknown,may
6,5,33,0,1,single,0,0,0,0,unknown,5,3.300000,1,-1,0,unknown,no,unknown,unknown,may


In [54]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [55]:
#update the configuration manager
from heart_disease_project.constants import * 
from heart_disease_project.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])
        data_transformation_config= DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        return data_transformation_config




In [212]:
# update the components
import os 
from sklearn.model_selection import train_test_split
from heart_disease_project import logger
import pandas as pd

class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config

    def set_the_header(self):
        df = pd.read_csv(self.config.data_path)
        df=df.iloc[1:]
        df.columns=df.iloc[0]
        df = df.drop(df.index[0])
        df.reset_index(drop=True)
        return df
    def feature_engineering(self):
        df = self.set_the_header()
        df[['job','education']]=df['jobedu'].str.split(',',expand=True)
        df[['months','years']]=df.month.str.split(",",expand=True)
        df.drop(['jobedu','month','years'],axis=1,inplace=True)
        df.targeted=df.targeted.map({'yes':1,'no':0})
        df.default=df.default.map({'yes':1,'no':0})
        df.loan=df.loan.map({'yes':1,'no':0})
        df.housing=df.housing.map({'yes':1,'no':0})
        seconds_mask=df['duration'].astype(str).str.endswith('sec')
        df.loc[seconds_mask,'duration']=df.loc[seconds_mask,'duration'].str.replace('sec', '').astype(float) / 60
        minutes_mask=df['duration'].astype(str).str.endswith('min')
        df.loc[minutes_mask,'duration']=df.loc[minutes_mask,'duration'].str.replace('min','').astype(float)
        df.duration=df.duration.astype(float)
        return df
    
    
    